In [1]:
# Import libraries
import os
import glob
import pandas as pd
from nilearn import plotting, image
from nilearn.glm.second_level import SecondLevelModel

## 1. Configuration
Define paths and subjects.

In [2]:
# Paths
first_level_dir = "first_level_results"
output_dir = "second_level_results"
os.makedirs(output_dir, exist_ok=True)

# List of subjects
subjects = [f"sub-{i:02d}" for i in range(1, 22)]
print(f"Subjects: {len(subjects)}")

Subjects: 21


## 2. Collect First-Level Maps
We collect the Z-maps for the contrast `incongruent_vs_congruent` generated in the first-level analysis.

In [3]:
contrast_id = "incongruent_vs_congruent"
input_maps = []

for sub in subjects:
    map_path = os.path.join(first_level_dir, f"{sub}_zmap_{contrast_id}.nii.gz")
    if os.path.exists(map_path):
        input_maps.append(map_path)
    else:
        print(f"Warning: Map not found for {sub} at {map_path}")

print(f"Found {len(input_maps)} maps for second-level analysis.")

Found 21 maps for second-level analysis.


## 3. Second-Level Model Fitting
Fit a One-Sample T-Test to determine group-level activation.

In [4]:
print("Fitting Second-Level Model...")

# Define model (One-Sample T-Test)
group_model = SecondLevelModel(
    smoothing_fwhm=8.0,  # Standard smoothing for group level
    minimize_memory=True
)

# Fit model
if not input_maps:
    raise ValueError("No input maps found. Please run the first-level analysis first.")

# Create design matrix (One-Sample T-Test)
# Following nilearn example: plot_second_level_one_sample_test.py
design_matrix = pd.DataFrame([1] * len(input_maps), columns=['intercept'])

group_model.fit(input_maps, design_matrix=design_matrix)

# Compute Group Contrast (intercept)
z_map_group = group_model.compute_contrast(output_type='z_score')

# Save result
group_out_path = os.path.join(output_dir, f"group_zmap_{contrast_id}.nii.gz")
z_map_group.to_filename(group_out_path)
print(f"Group Z-map saved to: {group_out_path}")

Fitting Second-Level Model...


ValueError: Field of view of image #1 is different from reference FOV.
Reference affine:
array([[   3.        ,    0.        ,    0.        ,  -96.66867828],
       [   0.        ,    3.        ,    0.        , -106.78915405],
       [   0.        ,    0.        ,    4.        ,  -81.61445618],
       [   0.        ,    0.        ,    0.        ,    1.        ]])
Image affine:
array([[  3.        ,   0.        ,   0.        , -80.04216766],
       [  0.        ,   3.        ,   0.        , -91.60843658],
       [  0.        ,   0.        ,   4.        , -80.89156342],
       [  0.        ,   0.        ,   0.        ,   1.        ]])
Reference shape:
(64, 64, 40)
Image shape:
(64, 64, 40, 1)


## 4. Generate Report
Create an HTML report for the group analysis.

In [ ]:
print("Generating Group Level Report...")
report = group_model.generate_report(
    contrasts="intercept",
    title="Group Level Result: Incongruent > Congruent"
)

report_path = os.path.join(output_dir, "report_group_level.html")
report.save_as_html(report_path)
print(f"Group report saved to: {report_path}")

## 5. Visualization
Plot the group-level statistical map.

In [ ]:
plotting.plot_stat_map(
    z_map_group, 
    title="Group Level: Incongruent > Congruent (Z > 3.0)", 
    threshold=3.0
)
plotting.show()